In [31]:
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np

In [32]:
df=pd.read_csv('sao-paulo-properties-april-2019.csv')

In [33]:
df.head()

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,District,Negotiation Type,Property Type,Latitude,Longitude
0,930,220,47,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.543138,-46.479486
1,1000,148,45,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.550239,-46.480718
2,1000,100,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.542818,-46.485665
3,1000,200,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.547171,-46.483014
4,1300,410,55,2,2,1,1,1,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.525025,-46.482436


In [34]:
df_sale=df[df['Negotiation Type']=='sale']
df_sale['m2']=np.round(df_sale['Price']+df_sale['Condo']/df_sale['Size'],2)
df_sale

C:\Users\General\AppData\Local\Temp\ipykernel_10412\1807620208.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sale['m2']=np.round(df_sale['Price']+df_sale['Condo']/df_sale['Size'],2)


,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,District,Negotiation Type,Property Type,Latitude,Longitude,m2
4901,732600,1000,74,1,2,1,2,1,0,1,0,Vila Madalena/São Paulo,sale,apartment,-23.552129,-46.692244,732613.51
4902,1990000,2400,164,4,5,2,3,1,1,1,0,Vila Madalena/São Paulo,sale,apartment,-23.551613,-46.699106,1990014.63
4903,720000,700,70,2,2,1,1,1,0,1,1,Vila Madalena/São Paulo,sale,apartment,-23.547687,-46.692594,720010.00
4904,1680000,1580,155,3,5,3,2,1,0,1,0,Vila Madalena/São Paulo,sale,apartment,-23.552590,-46.691104,1680010.19
4905,1200000,900,56,2,2,1,2,0,1,1,0,Vila Madalena/São Paulo,sale,apartment,-23.553087,-46.697890,1200016.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13635,265000,420,51,2,1,0,1,0,0,0,0,Jabaquara/São Paulo,sale,apartment,-23.653004,-46.635463,265008.24
13636,545000,630,74,3,2,1,2,0,0,1,0,Jabaquara/São Paulo,sale,apartment,-23.648930,-46.641982,545008.51
13637,515000,1100,114,3,3,1,1,0,0,1,0,Jabaquara/São Paulo,sale,apartment,-23.649693,-46.649783,515009.65
13638,345000,48,39,1,2,1,1,0,1,1,0,Jabaquara/São Paulo,sale,apartment,-23.652060,-46.637046,345001.23


In [35]:
df_sale['Price'].value_counts()

250000     97
240000     94
350000     93
320000     80
280000     80
           ..
281000      1
4182000     1
2199000     1
802800      1
559000      1
Name: Price, Length: 1409, dtype: int64

In [36]:
df_sale.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6412 entries, 4901 to 13639
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Price             6412 non-null   int64  
 1   Condo             6412 non-null   int64  
 2   Size              6412 non-null   int64  
 3   Rooms             6412 non-null   int64  
 4   Toilets           6412 non-null   int64  
 5   Suites            6412 non-null   int64  
 6   Parking           6412 non-null   int64  
 7   Elevator          6412 non-null   int64  
 8   Furnished         6412 non-null   int64  
 9   Swimming Pool     6412 non-null   int64  
 10  New               6412 non-null   int64  
 11  District          6412 non-null   object 
 12  Negotiation Type  6412 non-null   object 
 13  Property Type     6412 non-null   object 
 14  Latitude          6412 non-null   float64
 15  Longitude         6412 non-null   float64
 16  m2                6412 non-null   floa

In [37]:
novo_sale=df_sale.drop(['Price','New','District','Property Type','Negotiation Type','Latitude','Longitude'],axis=1).values
y_sale=df_sale['Price']

In [38]:
scaler = MinMaxScaler()
atributo_sale=scaler.fit(novo_sale)
sale_x=scaler.transform(novo_sale)
sale_x

array([[0.11210762, 0.07457627, 0.        , ..., 0.        , 1.        ,
        0.06935205],
       [0.2690583 , 0.22711864, 0.6       , ..., 1.        , 1.        ,
        0.19562258],
       [0.07847534, 0.06779661, 0.2       , ..., 0.        , 1.        ,
        0.06808638],
       ...,
       [0.12331839, 0.14237288, 0.4       , ..., 0.        , 1.        ,
        0.04749987],
       [0.00538117, 0.01525424, 0.        , ..., 1.        , 1.        ,
        0.03042731],
       [0.        , 0.02372881, 0.2       , ..., 0.        , 0.        ,
        0.01204869]])

In [39]:
X_train, X_test, y_train, y_test = train_test_split(novo_x,y,train_size=0.7)
X_train.shape


(4488, 10)

In [40]:
gradient=GradientBoostingRegressor()
gradient.fit(X_train,y_train)
pred=gradient.predict(X_test)
r2_score(y_test,pred)

0.9996300057968709

In [41]:
pickle.dump(gradient, open('Modelo_GradientBoostingRegressor.pkl', 'wb'))